In [29]:
import gensim
from gensim.models.doc2vec import LabeledSentence
import pandas as pd
import numpy as np
import os

In [9]:
df_ent = pd.read_csv('input/output.csv')
df_hack = pd.read_csv('input/HackerNoon.csv')
df_combined = pd.concat([df_ent, df_hack])
df_combined.dropna(inplace = True)
df_combined.reset_index(inplace =  True)
data = ' '.join(str(x) for x in df_combined['Text'])

In [7]:
import spacy
nlp = spacy.load('en')
sentences = []
sentences_label = []

def create_sentence(doc):
    punctuation = [".","?","!",":","…"]
    sentences = []
    sent = []

    for word in doc:
        if word.text not in punctuation:
            if word.text not in ("\n","\n\n",'\u2009','\xa0', " "):
                sent.append(word.text.lower())
        else:
            sent.append(word.text.lower())
            if len(sent) > 1:
                sentences.append(sent)
            sent=[]
    return sentences

In [16]:
myDoc = nlp(data)
sents = create_sentence(myDoc)
# print (sents[:10])
sentences = sentences + sents

In [17]:
for i in range(np.array(sentences).shape[0]):
    sentences_label.append("ID" + str(i))

In [23]:
for sentence, label in zip(sentences[:10], sentences_label[:10]):
    print (sentence[:10], label)

['i', 'hear', 'this', 'sentiment', 'so', 'often', 'from', 'creative', 'entrepreneurs', 'and'] ID0
['what', 'goes', 'in', 'and', 'what', 'comes', 'out', 'people', ',', 'resources'] ID1
['what', 'can', 'you', 'just', 'not', 'understand', '?'] ID2
['why', 'is', 'something', 'the', 'way', 'that', 'it', 'is', '?'] ID3
['why', 'is', 'n’t', 'it', 'better', '?'] ID4
['ask', 'yourself', 'these', 'questions', 'to', 'identify', 'the', 'problem', 'you', 'are'] ID5
['my', 'friends', 'and', 'i', 'created', 'a', '5-question', 'mini', '-', 'survey'] ID6
['randy', 'showed', 'the', 'idea', 'to', 'his', 'seal', 'team', 'and', 'then'] ID7
['those', 'that', 'have', 'to', 'do', 'with', 'the', 'market', ',', 'founders'] ID8
['”', 'asked', 'jackson', 'josh', 'nodded', 'enthusiastically', 'unbeknownst', 'to', 'the', 'duo'] ID9


In [24]:
class LabeledLineSentence(object):

    def __init__(self, doc_list, labels_list):

        self.labels_list = labels_list

        self.doc_list = doc_list

    def __iter__(self):

        for idx, doc in enumerate(self.doc_list):

            yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])

In [27]:
import time
def train_doc2vec_model(data, docLabels, size=300, sample=0.000001, dm=0, hs=1, window=10, min_count=0, workers=8,alpha=0.024, min_alpha=0.024, epoch=15, save_file='output/doc2vec.w2v') :
    
    startime = time.time()
    print("{0} articles loaded for model".format(len(data)))

    it = LabeledLineSentence(data, docLabels)

    model = gensim.models.Doc2Vec(size=size, sample=sample, dm=dm, window=window, min_count=min_count, workers=workers,alpha=alpha, min_alpha=min_alpha, hs=hs) # use fixed learning rate

    model.build_vocab(it)

    for epoch in range(epoch):

        print("Training epoch {}".format(epoch + 1))

        model.train(it,total_examples=model.corpus_count,epochs=model.iter)

        # model.alpha -= 0.002 # decrease the learning rate
        # model.min_alpha = model.alpha # fix the learning rate, no decay
    

    #saving the created model

    model.save(os.path.join(save_file))

    print('model saved')

In [30]:
train_doc2vec_model(sentences, sentences_label, size=800, sample=0.0, alpha=0.025, min_alpha=0.001, min_count=0, window=10, epoch=50, dm = 0, hs = 1)

102 articles loaded for model


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  del sys.path[0]


Training epoch 1


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


Training epoch 2
Training epoch 3
Training epoch 4
Training epoch 5
Training epoch 6
Training epoch 7
Training epoch 8
Training epoch 9
Training epoch 10
Training epoch 11
Training epoch 12
Training epoch 13
Training epoch 14
Training epoch 15
Training epoch 16
Training epoch 17
Training epoch 18
Training epoch 19
Training epoch 20
Training epoch 21
Training epoch 22
Training epoch 23
Training epoch 24
Training epoch 25
Training epoch 26
Training epoch 27
Training epoch 28
Training epoch 29
Training epoch 30
Training epoch 31
Training epoch 32
Training epoch 33
Training epoch 34
Training epoch 35
Training epoch 36
Training epoch 37
Training epoch 38
Training epoch 39
Training epoch 40
Training epoch 41
Training epoch 42
Training epoch 43
Training epoch 44
Training epoch 45
Training epoch 46
Training epoch 47
Training epoch 48
Training epoch 49
Training epoch 50
model saved


In [35]:
from six.moves import cPickle
d2v_model = gensim.models.doc2vec.Doc2Vec.load('output/doc2vec.w2v')

sentence_vector = []
t = 10

for i in range(len(sentences)):
    if i%t == 0 : 
        print("sentence", i, ":", sentences[i])
        print("***")
    sent = sentences[i]
    sentence_vector.append(d2v_model.infer_vector(sent, alpha = 0.025, min_alpha = 0.001, steps = 10000))
    
    


sentence 0 : ['i', 'hear', 'this', 'sentiment', 'so', 'often', 'from', 'creative', 'entrepreneurs', 'and', 'small', 'business', 'founders', 'entrepreneurs', 'by', 'nature', 'tend', 'to', 'be', 'people', 'who', 'have', 'big', 'dreams', 'and', 'the', 'guts', 'to', 'follow', 'them', 'on', 'the', 'flip', 'side', ',', 'they', 'often', 'shy', 'away', 'from', ',', 'look', 'down', 'upon', ',', 'or', 'downplay', 'the', 'details', 'that', 'go', 'along', 'with', 'execution', 'you', 'can', 'set', 'big', ',', 'hairy', ',', 'audacious', 'goals', 'for', 'your', 'company', 'goals', 'are', 'critical', 'you', 'can', 'also', 'determine', 'your', 'core', 'values', 'but', 'without', 'understanding', 'what', 'goes', 'into', 'the', 'execution', ',', 'you', 'risk', 'tanking', 'your', 'company', 'one', 'concept', 'to', 'use', 'for', 'a', 'framework', 'around', 'execution', 'is', 'the', 'concept', 'of', 'profitability', 'when', 'you', 'think', 'about', 'the', 'profitability', 'of', 'an', 'idea', ',', 'you', 'ne

sentence 60 : ['here', ',', 'you', 'can', 'see', 'that', 'the', 'model', 'has', 'learned', 'in', 'the', 'way', 'it', 'has', 'generated', 'the', 'paragraphs', 'and', 'sentences', 'with', 'appropriate', 'spacing', 'it', 'still', 'lacks', 'perfection', 'and', 'also', 'does', 'n’t', 'make', 'sense', 'seeing', 'signs', 'of', 'success', '\u200a', '—', '\u200a', 'the', 'first', 'task', 'is', 'to', 'create', 'a', 'model', 'that', 'can', 'learn', ',', 'and', 'then', 'the', 'second', 'one', 'is', 'used', 'to', 'improve', 'on', 'that', 'model', 'this', 'can', 'be', 'obtained', 'by', 'training', 'the', 'model', 'with', 'a', 'larger', 'training', 'dataset', 'and', 'longer', 'training', 'durations', 'if', 'you', 'found', 'this', 'article', 'interesting', ',', 'you', 'can', 'explore', 'python', 'deep', 'learning', 'projects', 'to', 'master', 'deep', 'learning', 'and', 'neural', 'network', 'architectures', 'using', 'python', 'and', 'keras', 'python', 'deep', 'learning', 'projects', 'imparts', 'all', '

TypeError: Required argument 'file' (pos 2) not found

In [38]:
sentences_vector_file = os.path.join('output', 'sentences_vector_800_a025_ma001_s10000.pkl')

with open(os.path.join(sentences_vector_file), 'wb') as f:
    cPickle.dump(sentence_vector, f)

In [42]:
nb_sequenced_sentences = 15
vector_dim = 500

X_train = np.zeros((len(sentences), nb_sequenced_sentences, vector_dim), dtype = np.float)
y_train = np.zeros((len(sentences), vector_dim), dtype = np.float)

t = 10
for i in range(len(sentences) - nb_sequenced_sentences - 1):
    if i%t == 0:
        print('new sequence: ', i)
    
    for k in range(nb_sequenced_sentences):
        sent = sentences_label[i+k]
        vect = sentences_label[i+k]
        
        if i%t == 0:
            print (' ', k+1 ,'th vector for this sequence. Sentence ', sent, "(vector dim = ", len(vect), ")")
        for j in range(len(vect)):
            X_train[i, k, j] = vect[j]
    senty = sentences_label[i + nb_sequenced_sentences]
    vecty = sentence_vector[i + nb_sequenced_sentences]
    if i%t == 0:
        print(' y vector for this sequence ', senty, ': (vector dim = ', len(vecty), ')')
    for j in range(len(vecty)):
        y_train[i, j] = vecty[j]
print(X_train.shape, y_train.shape)

new sequence:  0
  1 th vector for this sequence. Sentence  ID0 (vector dim =  3 )


ValueError: could not convert string to float: 'I'